In [1]:
#load os
import os

#load Flask 
import flask
from flask import Flask, render_template, request
from flask_uploads import UploadSet, configure_uploads, IMAGES

#load model preprocessing and needed keras packa
import numpy as np
import pandas as pd

import sys

import cv2
import scipy
import skimage
from skimage.transform import resize

import keras.models
from keras.models import model_from_json
from keras.layers import Input

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#initialize app
app = flask.Flask(__name__)

In [3]:
# Load pre-trained model into memory
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#load weights into new model
loaded_model.load_weights("weights.h5")
print("Loaded Model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded Model from disk


https://github.com/anagar20/Resnet-Image-Classification-Flask-App

Note: I referred to the code above as a starting point

In [4]:
photos = UploadSet('photos', IMAGES)
app.config['UPLOADED_PHOTOS_DEST'] = '.'
configure_uploads(app, photos)

In [5]:
@app.route('/', methods=['GET','POST'])
def upload():
    if request.method == 'POST':           
        #Delete output.png if already exists (will be overwriting this and get error if not removed)
        if os.path.exists('output.png')==True:
            os.remove('output.png')
        
        # save file to network (note: run line below if you don't want to keep images)
        filename = photos.save(request.files['photo'])
        # rename file so you don't blow up storage with files uploaded
        os.rename('./'+filename,'./'+'output.png')

        # convert to matrix that is size needed for CNN
        img_matrix = cv2.imread('output.png')
        img_matrix_downsampled = skimage.transform.resize(img_matrix, (256,256,3)) 
        img_matrix_resized = img_matrix_downsampled.reshape(1,3,256,256)
       
        # put through pre-trained CNN and send prediction to HTML to give user response
        pred_df = pd.DataFrame(loaded_model.predict(img_matrix_resized))[1]
        if pred_df.iloc[0]<0.50:
            #Note: these numbers are based on model precision in test sample
            prediction = "Not at high risk of pneumonia. Rescan if symptoms persist"
        else:
            prediction = "At high risk of pneumonia. Please provide treatment."

        return render_template('results_page.html', prediction=prediction)
    
    else:
        #load upload page
        return render_template('upload_page.html')

In [ ]:
if __name__ == "__main__":
    # start the flask app, allow remote connections
    #decide what port to run the app in
    port = int(os.environ.get('PORT', 5000))
    #run the app locally on the givn port
    app.run(host='0.0.0.0', port=port)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jul/2019 23:02:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2019 23:03:43] "POST / HTTP/1.1" 200 -
